In [10]:
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions


embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large-instruct"
)

client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection(
    name="documents_collection",
    embedding_function=embedding_model
)



In [11]:
import os
import json
import numpy as np


files = os.listdir('/home/wagyu0923/project/Document_Analyzer/embedded_files')

ids = []
meta_datas = []
embeddings = []
chunks = []

for file in files:
    with open(f'/home/wagyu0923/project/Document_Analyzer/embedded_files/{file}', 'r') as f:
        embedded_text = json.load(f)
    
    
    for idx, data in enumerate(embedded_text):
        meta_data = {}
        meta_data['source'] = file
        meta_data['page'] = data['page']
        meta_datas.append(meta_data)

        chunks.append(data['chunk'])

        embeddings.append(data['embedding'])

        ids.append(f'{file}_chunk_{idx}')


collection.add(
    embeddings = embeddings,
    metadatas = meta_datas,
    ids = ids,
    documents=chunks
)


In [3]:
query = input()

collected_data = collection.query(
    query_texts= [query],
    n_results=7,
)
retrived_data = ''

for meta, chunk in zip(collected_data['metadatas'][0], collected_data['documents'][0]):
    source = meta['source']
    page = meta['page']
    retrived_data += f'source : {source}, page : {page}, \n content : {chunk} \n\n'

In [ ]:
import ollama

OLLAMA_MODEL = 'gpt-oss:20b'
SYSTEM_PROMPT = f"""You are an expert financial analyst. Your goal is to provide a clear and well-supported answer based *only* on the provided context.

**Instructions:**
1.  **Analyze and Synthesize:** Carefully analyze the user's QUESTION and the provided CONTEXT. Synthesize the information to infer a comprehensive answer.
2.  **Cite Sources:** For every piece of information or claim in your answer, you **must cite the specific `source` and `page`** it came from. For example: "The company was fined 2.7 billion won (Source: filename.json, Page: 214)."
3.  **Adhere to Context:** Base your answer **strictly** on the information within the CONTEXT. Do not use any outside knowledge.
4.  **Handle Missing Information:** If the CONTEXT does not contain enough information to answer the question, state that you cannot provide an answer based on the given documents.
5.  **Language:** Respond in Korean.

**CONTEXT:** {retrived_data}

**QUESTION:**
{query}
"""

response = ollama.chat(
    model=OLLAMA_MODEL,
    messages=[
        {'role': 'user', 'content': SYSTEM_PROMPT}
    ],
    options={

        'temperature': 0.0,
        'repeat_penalty': 1.1,
        'seed' : 42
    }
)
outputs = response['message']['content']

In [9]:
print(outputs)

**세토피아의 투자 리스크 요약**

| 리스크 항목 | 구체적 내용 | 근거 |
|-------------|-------------|------|
| **규제·법적 제재** | • 2025년 1월 20일, 증권선물위원회 과징금 1억 7,680만원(상장법인 신고·공시 의무 위반) <br>• 2025년 3월 12일, 증권선물위원회 과징금 27억원·과태료 1억원(회계장부·재무제표 반영 미흡 등) <br>• 2025년 3월 31일, 금융감독원 과태료 1,000만원(외국환거래법 위반) <br>• 2025년 4월 28일, 증권선물위원회 증권발행제한 6개월(2025‑04‑28~2025‑10‑27) | (Source: 세토피아 반기보고서, page 214) |
| **법적 소송·손해배상** | • 서울중앙지방법원 1심 손해배상 730,000원 <br>• 2심 전환사채금 2,180,000원 | (Source: 세토피아 반기보고서, page 124) |
| **보고서 제출 지연** | • 2019 사업보고서 제출 기한(2020‑05‑15) 이후 10일 지연(2020‑05‑25) | (Source: 세토피아 반기보고서, page 214) |
| **재무 성과 부진** | • 2025년 1분기(전기 대비) 기본주당이익(손실) -354원, 희석주당이익(손실) -354원 <br>• 2024년 1분기(전기 대비) 기본주당이익(손실) -358원, 희석주당이익(손실) -358원 | (Source: 세토피아 반기보고서, page 70) |
| **자본 구조 및 지분 구조** | • 비지배지분이 없으며, 지배기업 소유주 지분이 5,391,891,821원(전기 대비 26,894,275,816원)으로 변동이 큼 | (Source: 세토피아 반기보고서, page 70) |

### 주요 리스크 해석

1. **규제·법적 제재**  
   - 다수의 과징금·과태료가 부과되었으며, 증권발행제한이 6개월간 적용되는 등 규제 위험이 지속적으로 존재합니다. 이는 자본 조달 및 주가에 부정적 영향을 미칠 수

In [8]:
print(retrived_data)

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 214, 
 content : 료 1,000만원 외국환거래법 위반 외국환거래법 제18조 및 외국환거래규정 제7-31조 20250428 증권선물위원회 (주)세토피아 증권발행제한 6개월 (20250428~20251027) - 2019년 사업보고서 지연 제출 - 제출기한 : 20200515 - 제출일자 : 20200525 자본시장과 금융투자업에 관한 법률 제159조 등 제재 조치일 조치기관 조치대상자 조치내용 사유 및 근거법령 조치에 대한 이행현황 재발방지를 위한 대책 2023- 

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 124, 
 content :  (주)세토피아 서울중앙지방법원 730,000 손해배상 1심 계류중 박** (주)세토피아 서울중앙지방법원 2,180,000 전환사채금 2심 계류중(*) 

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 214, 
 content :  제재기관 대상자 처벌 또는 조치 내용 금전적 제재금액 사유 근거법령 20250120 (*1) 증권선물위원회 (주)세토피아 과징금 과징금 1억7,680만원 상장법인 등의 신고·공시의무 위반 자본시장과 금융투자업에 관한 법률 제161조 등 20250312 (*2) 증권선물위원회 (주)세토피아 1 과징금 2 과태료 3 감사인지정 3년 4 면직권고 5 지적사항 회계장부 및 재무제표 반영 6 검찰통보 1 과징금 27억원 2 과태료 1억원 1 금융자산· 

source : [세토피아][정정]반기보고서(2025.09.09)_chunk_embedding.json, page : 214, 
 content : 지정 3년 4 면직권고 5 지적사항 회계장부 및 재무제표 반영 6 검찰통보 1 과징금 27억원 2 과태료 1억원 1 금융자산·부채 과대